In [60]:
import numpy as np
import pandas as pd
import math

In [2]:
pd.options.display.max_columns = 50

In [3]:
# Methods
def unpack_column(dataFrame,col_name):
    newDf = dataFrame.from_records(dataFrame[col_name].dropna().tolist())
    return newDf

def unpack_and_destroy(dataFrame,column):
    temp = unpack_column(dataFrame,column)
    for col in temp.columns:
        dataFrame[col] = temp[col]
    #dataFrame = dataFrame.drop(columns=[column])
    return dataFrame

def sort_db_value(db,value,asc=False):
    return db.sort_values(by = value, ascending = asc)

def freq_array(freq_list):
    freq = []
    for i in freq_list:
        freq.append(i)
    return freq

def set_labels(freq, ax,horz_shift=.2,vert_shift=1000,w="bold",fsize=10,c="b"):
    for i, v in enumerate(freq):
        ax.text(i-horz_shift,v+vert_shift, str(round(v,2)), weight = w,fontsize=fsize,color = c)
        
def wrap_axis_labels(labels_list,wrap_length=13):
    labels_list = [ '\n'.join(wrap(l, wrap_length)) for l in labels_list]
    return labels_list

In [13]:
def zip_five(df):
    '''
    Updates df with a "ZIP_FIVE" column (5 digit zip code)
    
    Parameters
    ----------
    df : Pandas DataFrame with a "ZIP" column
    
    Returns
    -------
    Pandas DataFrame with a ZIP_FIVE column (5 digit zip code)
    '''
    df['ZIP_FIVE'] = df['ZIP'].apply(lambda x: x[:5])
    return df

In [14]:
def find_same_zip(df, zip_code):
    '''
    Looks through Database and returns Entries with same zip code
    
    Parameters
    ----------
    df : Pandas DataFrame of Charities
    zip_code : Zip Code to Match on
    
    Returns
    -------
    Pandas DataFrame with Entries matching provided zip code
    '''
    zip_code = str(zip_code)
    
    temp_df = df[all_regions_df['ZIP_FIVE'] == zip_code]
    temp_df_two = df[all_regions_df['ZIP'] == zip_code]
    temp_df = pd.concat((temp_df,temp_df_two))
    return temp_df

In [63]:
def create_NTEE_dict(file):
    '''
    Create NTEE Dictionary from csv file
    
    Parameters
    ----------
    file : CSV file where first column is NTEE code
        and second column is NTEE code description
    
    Returns
    -------
    dictionary mapping NTEE code to NTEE code description
    '''
    NTEE_dict = dict()
    ntee_df = pd.read_csv(file,header=None)
    ntee_df = ntee_df.loc[:,:1]
    
    for index, row in ntee_df.iterrows():
        if type(row.loc[0]) != float:
            NTEE_dict[row.loc[0]] = row.loc[1].rstrip()
    
    return NTEE_dict

In [88]:
def create_major_categories_list(ntee_dict):
    '''
    Creates a list containing the major categories of NTEE codes
    
    Parameters
    ----------
    ntee_dict : Dictionary of NTEE codes and categories
    
    Returns
    -------
    list of major categories of NTEE codes
    '''
    major_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    major_categories = []

    for let in major_letters:
        major_categories.append(ntee_dict[let])
    
    return major_categories

In [208]:
def create_NTEE_Categories(df,ntee_dict):
    '''
    Create NTEE_Major_Category and NTEE_Minor_Category and column for a Pandas DataFrame
    
    Parameters
    ----------
    df : Pandas Dataframe containing "NTEE_CD" Column
    ntee_dict : Dictionary containing NTEE Codes as Keys and
        NTEE Categories as Values
    
    Returns
    -------
    Pandas Dataframe with "NTEE_Major_Category" and "NTEE_Minor_Category" column
    '''
    major_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    df['NTEE_Major_Category'] = df['NTEE_CD'].apply(lambda x: ntee_dict[x[0]] if \
                                                    (type(x) == str and x[0] in major_letters) else x)
    
    df['NTEE_Minor_Category'] = df['NTEE_CD'].apply(lambda x: ntee_dict[x[:3]] if \
                                                    (type(x) == str and x[:3] in ntee_dict.keys()) else x)
    
    return df

In [209]:
def find_major_category(df,category:str):
    '''
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Major_Category']==category]
    
    return temp_df

In [210]:
def search_by_zip_and_major_category(df, zipcode, category):
    '''
    
    '''
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_major_category(temp_df,category)
    
    return temp_df

In [211]:
def find_minor_category(df,category:str):
    '''
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Minor_Category']==category]
    
    return temp_df

In [212]:
def search_by_zip_and_minor_category(df, zipcode, category):
    '''
    
    '''
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_minor_category(temp_df,category)
    
    return temp_df

In [176]:
region1_df = pd.read_csv('Data/eo1.csv')
region2_df = pd.read_csv('Data/eo2.csv')
region3_df = pd.read_csv('Data/eo3.csv')
all_regions_df = pd.concat([region1_df,region2_df,region3_df])

In [177]:
all_regions_df = all_regions_df[all_regions_df["DEDUCTIBILITY"] == 1]

In [248]:
# all_regions_df = all_regions_df.drop(\
#     columns=['FILING_REQ_CD',"PF_FILING_REQ_CD","STATUS","NTEE_CD"])

In [178]:
all_regions_df = zip_five(all_regions_df)

In [179]:
ntee_dict = create_NTEE_dict('Data/NTEE_file - Sheet1.csv')

In [180]:
ntee_list = create_major_categories_list(ntee_dict)

In [182]:
all_regions_df = create_NTEE_Categories(all_regions_df,ntee_dict)

In [183]:
all_regions_df

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
0,19818,PALMER SECOND BAPTIST CHURCH,NaN,1050 THORNDIKE ST,PALMER,MA,01069-1507,3125,3,9,7000,195504,1,10,1000000,5,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,3514,01069,NaN,NaN
1,29215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,7000,197210,1,10,1000000,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN,02127,NaN,NaN
2,587764,IGLESIA BETHESDA INC,NaN,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,1,10,0,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,NaN,01852,"Religion-Related, Spiritual Development",Protestant
3,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,NaN,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,1,10,0,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,NaN,01840,"Religion-Related, Spiritual Development",Protestant
4,765634,MERCY CHAPEL INTERNATIONAL,NaN,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,1,10,0,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,NaN,02126,"Religion-Related, Spiritual Development",Christian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817691,996087839,KEIKI KOHOLA PROJECT,% RACHEL CARTWRIGHT,1330 SABAL LAKES RD,DELRAY BEACH,FL,33445-1290,0,3,3,1000,200107,1,4,0,1,1,201712.0,2,2,0,3,12,10109.0,22000.0,NaN,U50,CARTWRIGHT RACHAEL,33445,"Science and Technology Research Institutes, Se...","Biological, Life Science Research"
817692,996088748,UNOYO KOJIMA TRUST,% PACIFIC CENTURY TRUST,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200109,1,4,0,1,1,201803.0,4,3,0,1,3,377564.0,64829.0,NaN,T22,PACIFIC CENTURY TR TTEE,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Private Independent Foundations
817693,996089401,TOYO SAKUMOTO CHARITABLE TR,% PACIFIC CENTURY TRUST,PO BOX 3170,HONOLULU,HI,96802-3170,0,3,3,1000,200110,1,4,0,5,1,201903.0,5,3,0,1,3,660843.0,84812.0,NaN,T90,NaN,96802,"Philanthropy, Voluntarism and Grantmaking Foun...",Named Trusts/Foundations N.E.C.
817694,996165005,INDEPENDENT ORDER OF ODD FELLOWS,% DEBRA LA VERGNE,PO BOX 76,CUPERTINO,CA,95015-0076,28,8,9,1000,197211,1,0,36382265,5,1,201812.0,0,0,2,0,12,0.0,0.0,0.0,NaN,145 PHILOTESIAN REBEKAH LODGE,95015,NaN,NaN


In [185]:
manhasset_df = find_same_zip(all_regions_df,11030)

In [196]:
find_major_category(manhasset_df,'Arts, Culture and Humanities')

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
50021,112688348,ONDERDONK LANDMARK SOCIETY LTD,% ONDERDONK LANDMARK SOCIETY,1471 NORTHERN BLVD,MANHASSET,NY,11030-3049,0,3,3,1000,201209,1,15,62000000,1,1,201812.0,0,0,2,0,12,0.0,0.0,0.0,A80,NaN,11030,"Arts, Culture and Humanities","Historical Societies, Related Historical Activ..."
55804,113377867,CHINESE AMERICAN ASSOCIATION OF NORTH HEMPSTEA...,NaN,PO BOX 523,MANHASSET,NY,11030-0523,0,3,3,2100,199810,1,16,119040093,1,1,201812.0,3,3,1,0,12,83532.0,62939.0,42056.0,A230,NaN,11030,"Arts, Culture and Humanities","Cultural, Ethnic Awareness"
56266,113417966,METROPOLITAN YOUTH ORCHESTRA SOCIETY OF NEW YO...,% ANTHEA JACKSON,13 MYRTLE ST,MANHASSET,NY,11030-2236,0,3,3,2000,199906,1,15,0,1,1,201806.0,4,5,1,0,6,179027.0,878127.0,878127.0,A69,NaN,11030,"Arts, Culture and Humanities",Symphony Orchestras
57678,113532550,ART GUILD OF PORT WASHINGTON INC,NaN,200 PT WA BLVD,MANHASSET,NY,11030-1713,0,3,3,2000,200111,1,16,0,1,1,201712.0,3,4,1,0,12,86119.0,150062.0,150062.0,A25,NaN,11030,"Arts, Culture and Humanities",Arts Education
59795,116037632,MANHASSET ART ASSOCIATION INC,% ANGELA TORO TREASURER,PO BOX 36,MANHASSET,NY,11030-0036,0,3,3,1000,201705,1,4,119059000,1,1,NaN,0,0,0,1,12,NaN,NaN,NaN,A26,NaN,11030,"Arts, Culture and Humanities",Arts Council/Agency
59876,116038976,SCIENCE MUSEUM OF LONG ISLAND,% ROSEDALE & SCERBO CPAS,1526 PLANDOME RD,MANHASSET,NY,11030-1038,0,3,3,2000,196411,1,16,59000000,1,1,201808.0,0,0,2,0,8,0.0,0.0,0.0,A520,NaN,11030,"Arts, Culture and Humanities",Children's Museums
71364,133936809,MELLIJOR CHARITABLE FOUNDATION INC,% WILLIAM BORDEN,220 ESTATES TER S,MANHASSET,NY,11030-4027,0,3,3,1000,199806,1,3,60000000,1,1,201712.0,1,1,0,1,12,1.0,1.0,NaN,A50Z,NaN,11030,"Arts, Culture and Humanities","Museum, Museum Activities"
168938,320476954,BELARUSIAN KRECEUSKI FOUNDATION,% VALERY DVORNIK,172 MILL SPRING RD,MANHASSET,NY,11030-3618,0,3,3,2000,201602,1,4,0,1,1,201712.0,4,2,0,1,12,134937.0,17851.0,NaN,A50,NaN,11030,"Arts, Culture and Humanities","Museum, Museum Activities"
179487,453843315,CREATIVE ARTS AS A GLOBAL RESOURSE INC,% WAYNE MIRSKY,1010 NORTHERN BLVD NO 208,MANHASSET,NY,11030-0000,0,3,3,1200,201306,1,15,0,1,1,201712.0,1,2,2,0,12,1738.0,16281.0,16281.0,A99,NaN,11030,"Arts, Culture and Humanities","Arts, Culture, and Humanities N.E.C."
187632,462941840,APPLE SEED FOUNDATION INC,NaN,47 PLANDOME RD,MANHASSET,NY,11030-2330,0,3,3,1000,201802,1,4,0,1,1,201712.0,1,2,0,1,12,9692.0,22000.0,NaN,A80,NaN,11030,"Arts, Culture and Humanities","Historical Societies, Related Historical Activ..."


In [216]:
search_by_zip_and_major_category(all_regions_df,zipcode=11030,category='Education').head(5)

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
49432,112615673,THE MANHASSET SCHOOL COMMUNITY ASSOCIATION INC,NaN,PO BOX 33,MANHASSET,NY,11030-0033,0,3,3,1000,198210,1,15,35000000,1,1,201808.0,5,5,1,0,8,864181.0,892914.0,753261.0,B20I,NaN,11030,Education,"Elementary, Secondary Education, K - 12"
50426,112741980,MANHASSET AFTER SCHOOL EXPERIENCE INC,% MUNSEY PARK SCHOOL,C/O MUNSEY PARK SCHOOL 1 HUNT LANE,MANHASSET,NY,11030-0000,0,3,3,1000,198712,1,16,574000000,1,1,201806.0,4,5,1,0,6,449513.0,587798.0,587798.0,B900,NaN,11030,Education,Educational Services and Schools - Other
55881,113384115,EWHA WOMANS UNIVERSITY ALUMNAE ASSOCIATION OF ...,% EDWARD M HA,47 STONE HILL DR S,MANHASSET,NY,11030-4427,0,3,3,1000,199807,1,15,994034000,1,1,201802.0,4,4,1,0,2,176077.0,148024.0,148024.0,B40,NaN,11030,Education,Higher Education Institutions
58737,113633067,THE MATTHEW G VIANNA MEMORIAL FUND 10-17-01,% MARILYNDA VIANNA,79 PARKWOODS RD,MANHASSET,NY,11030-1538,0,3,3,1000,200204,1,16,0,2,1,201812.0,3,1,2,0,12,27595.0,1150.0,1150.0,B82,NaN,11030,Education,"Scholarships, Student Financial Aid Services, ..."
60007,116048468,MANHASSET STUDENT AID ASSOCIATION INC,NaN,PO BOX 93,PLANDOME,NY,11030-0093,0,3,3,2000,196606,1,15,59561040,1,1,201806.0,4,3,2,0,6,294914.0,38526.0,38526.0,B82Z,NaN,11030,Education,"Scholarships, Student Financial Aid Services, ..."


In [223]:
search_by_zip_and_major_category(all_regions_df,zipcode=97133,category='Education')

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
314357,474078112,ELEVATE A GIRL FOUNDATION INC,NaN,PO BOX 407,NORTH PLAINS,OR,97133-0407,0,3,3,1000,201506,1,15,0,1,1,201812.0,0,0,2,0,12,0.0,0.0,0.0,B82,NaN,97133,Education,"Scholarships, Student Financial Aid Services, ..."
449779,680518096,FRIENDS OF THE NORTH PLAINS PUBLIC LIBRARY,% CHRISTINE M LANTZ,PO BOX 525,NORTH PLAINS,OR,97133-0525,0,3,3,2000,200309,1,16,0,1,1,201806.0,3,2,2,0,6,34116.0,23534.0,23534.0,B117,NaN,97133,Education,Single Organization Support
757052,931067277,NORTH PLAINS P T O,% TREASURER,32030 NW NORTH AVE,NORTH PLAINS,OR,97133-6146,0,3,3,1000,200201,1,16,0,1,1,201805.0,0,0,2,0,5,0.0,0.0,0.0,B94,NaN,97133,Education,Parent/Teacher Group


In [224]:
ntee_list

['Arts, Culture and Humanities',
 'Education',
 'Environmental Quality, Protection and Beautification',
 'Animal-Related',
 'Health - General and Rehabilitative',
 'Mental Health, Crisis Intervention',
 'Diseases, Disorders, Medical Disciplines',
 'Medical Research',
 'Crime, Legal-Related',
 'Employment, Job-Related',
 'Food, Agriculture and Nutrition',
 'Housing, Shelter',
 'Public Safety, Disaster Preparedness and Relief',
 'Recreation, Sports, Leisure, Athletics',
 'Youth Development',
 'Human Services - Multipurpose and Other',
 'International, Foreign Affairs and National Security',
 'Civil Rights, Social Action, Advocacy',
 'Community Improvement, Capacity Building',
 'Philanthropy, Voluntarism and Grantmaking Foundations',
 'Science and Technology Research Institutes, Services',
 'Social Science Research Institutes, Services',
 'Public, Society Benefit - Multipurpose and Other',
 'Religion-Related, Spiritual Development',
 'Mutual/Membership Benefit Organizations, Other',
 'Unk

In [227]:
search_by_zip_and_major_category(all_regions_df,10017,'Youth Development')

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
3906,10718335,AJP FOUNDATION FOR CHILDREN INC,% PERGAMENT ADVISORS LLC,780 3RD AVENUE 9TH FLOOR,NEW YORK,NY,10017-2024,0,3,3,1000,200310,1,4,0,1,1,201712.0,2,1,0,1,12,16508.0,11.0,NaN,O12,ALL FOR KIDS,10017,Youth Development,Fund Raising and/or Fund Distribution
153136,263884626,MOMENTUM TEENS FOR LEADERSHIP CORP,% ROBIN LYNN,228 EAST 45TH STREET 4TH FLOOR,NEW YORK,NY,10017-3303,0,3,3,1000,200912,1,15,0,1,1,201612.0,2,3,1,0,12,13997.0,98750.0,98750.0,O500,NaN,10017,Youth Development,"Youth Development Programs, Other"
172924,371625465,JOHNNY MAC TENNIS PROJECT INC,% SPORT TIME,529 FIFTH AVE SUITE 901,NEW YORK,NY,10017-4608,0,3,3,1000,201206,1,15,0,1,1,201712.0,4,5,1,0,12,216243.0,522520.0,522520.0,O99,NaN,10017,Youth Development,Youth Development N.E.C.
201508,474427793,YUVA UNSTOPPABLE USA INC,% AMITABH SHAH,240 E47TH STREET,NEW YORK,NY,10017-2131,0,3,3,1000,201511,1,16,0,1,1,201812.0,2,4,1,0,12,19913.0,130788.0,130788.0,O99,NaN,10017,Youth Development,Youth Development N.E.C.
228745,815459134,MSBGC-NYC SUPPORT CORPORATION,% BARRY BREGMAN,733 THIRD AVE FLOOR 2,NEW YORK,NY,10017-3204,0,3,3,1000,201704,1,21,0,1,1,201709.0,8,1,2,0,9,45200965.0,5890.0,5890.0,O11,NaN,10017,Youth Development,Single Organization Support
229015,820588686,STAY-FOCUSED INC,% ROGER MULLER,245 E 44TH ST APT 22B,NEW YORK,NY,10017-4342,0,3,3,1000,200308,1,15,0,1,1,201712.0,3,4,1,0,12,48240.0,209958.0,194212.0,O50,NaN,10017,Youth Development,"Youth Development Programs, Other"
245325,836651902,ELLIS FAMILY FOUNDATION,NaN,CARE OF TAUB-355 LEXINGTON AVE FL 8,NEW YORK,NY,10017-0000,0,3,3,1000,201903,1,4,0,2,1,NaN,0,0,0,1,12,NaN,NaN,NaN,O53,FREDDIE L ELLIS TTEE,10017,Youth Development,Youth Development - Business


## Rate By:
1. Narrow Down Categories
2. Charity Navigator Rating
3. Asset/Income
4. Region/State from ZipCode

In [228]:
from uszipcode import SearchEngine

In [229]:
search = SearchEngine(simple_zipcode=True)

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [243]:
search.by_zipcode(10017)

SimpleZipcode(zipcode='10017', zipcode_type='Standard', major_city='New York', post_office_city='New York, NY', common_city_list=['New York'], county='New York County', state='NY', lat=40.75, lng=-73.973, timezone='Eastern', radius_in_miles=0.6818181818181818, area_code_list=['212', '646', '718'], population=16575, population_density=51775.0, land_area_in_sqmi=0.32, water_area_in_sqmi=0.0, housing_units=12875, occupied_housing_units=10763, median_home_value=637100, median_household_income=100652, bounds_west=-73.981822, bounds_east=-73.964271, bounds_north=40.757912, bounds_south=40.747251)

In [232]:
ntee_list

['Arts, Culture and Humanities',
 'Education',
 'Environmental Quality, Protection and Beautification',
 'Animal-Related',
 'Health - General and Rehabilitative',
 'Mental Health, Crisis Intervention',
 'Diseases, Disorders, Medical Disciplines',
 'Medical Research',
 'Crime, Legal-Related',
 'Employment, Job-Related',
 'Food, Agriculture and Nutrition',
 'Housing, Shelter',
 'Public Safety, Disaster Preparedness and Relief',
 'Recreation, Sports, Leisure, Athletics',
 'Youth Development',
 'Human Services - Multipurpose and Other',
 'International, Foreign Affairs and National Security',
 'Civil Rights, Social Action, Advocacy',
 'Community Improvement, Capacity Building',
 'Philanthropy, Voluntarism and Grantmaking Foundations',
 'Science and Technology Research Institutes, Services',
 'Social Science Research Institutes, Services',
 'Public, Society Benefit - Multipurpose and Other',
 'Religion-Related, Spiritual Development',
 'Mutual/Membership Benefit Organizations, Other',
 'Unk

In [234]:
all_regions_df.groupby('NTEE_Major_Category').size().sort_values(ascending=False)

NTEE_Major_Category
Religion-Related, Spiritual Development                  150613
Education                                                140807
Human Services - Multipurpose and Other                  113278
Philanthropy, Voluntarism and Grantmaking Foundations     91159
Arts, Culture and Humanities                              90549
Recreation, Sports, Leisure, Athletics                    70233
Youth Development                                         45569
Health - General and Rehabilitative                       37267
Community Improvement, Capacity Building                  30662
Animal-Related                                            29743
Housing, Shelter                                          22937
Environmental Quality, Protection and Beautification      21344
International, Foreign Affairs and National Security      21160
Public, Society Benefit - Multipurpose and Other          18815
Public Safety, Disaster Preparedness and Relief           16830
Mental Health, Crisi

In [237]:
search_by_zip_and_major_category(all_regions_df,11030,'Animal-Related')

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
48687,112518200,AMERICAN ASSOCIATION FOR LABORATORY ANIMAL SCI...,NaN,350 COMMUNITY DR,MANHASSET,NY,11030-3816,0,3,3,1000,198109,1,16,995205000,1,1,201806.0,4,3,1,0,6,155406.0,58280.0,12439.0,D034,METROPOLITAN NY BRANCH,11030,Animal-Related,"Professional Societies, Associations"
171914,364695500,A LONG ROAD HOME INC,% DEBORAH RISTAU,PO BOX 1395,MANHASSET,NY,11030-6395,0,3,3,1000,201203,1,15,0,1,1,201812.0,2,3,1,0,12,10182.0,83777.0,83777.0,D20,NaN,11030,Animal-Related,Animal Protection and Welfare
224641,813539432,FLUTTERBY FOUNDATION INC,NaN,47 PLANDOME RD,MANHASSET,NY,11030-2330,0,3,3,1000,201705,1,15,0,1,1,201812.0,0,0,2,0,12,0.0,0.0,0.0,D30,NaN,11030,Animal-Related,"Wildlife Preservation, Protection"


In [246]:
x = all_regions_df.loc[1]

In [247]:
x

,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category
1,29215,ST GEORGE CATHEDRAL,NaN,523 E BROADWAY,SOUTH BOSTON,MA,02127-4415,2365,3,9,7000,197210,1,10,1000000,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN,02127,NaN,NaN
1,10036381,DISABLED AMERICAN VETERANS,% BILLIENITA CARDONA,15623 S CENTERLINE RD,RUDYARD,MI,49780-9349,557,4,9,3000,194206,1,0,908279265,5,1,201806.0,0,0,2,0,6,0.0,0.0,0.0,NaN,93 STRAITS,49780,NaN,NaN
1,360268,IGLESIA VICTORIA,NaN,1445 N BOONVILLE AVE,SPRINGFIELD,MO,65802-1894,1678,3,9,7000,196408,1,10,1002029,5,1,NaN,0,0,6,0,12,NaN,NaN,NaN,NaN,NaN,65802,NaN,NaN


In [ ]:
x['city'] = x['ZIP_FIVE'].apply